# Largest Value Path of Graph (Google)
##### *Algorithms & Data Structures*

In a directed graph, each node is assigned an uppercase letter. We define a path’s value as the number of the most frequently-occurring letter along that path.

Given a graph with `n` nodes and `m` directed edges, return the largest value path of the graph.

The graph is represented with a string and an edge list. The `i`th character represents the uppercase letter of the `i`th node. Each tuple in the edge list `(i, j)` means there is a directed edge from the `i`th node to the `j`th node. Self edges and multi-edges are possible.

### Solution

The naive solution here would be to try every single path from every vertex, count up each path's value and keep track of the maximum value we've seen. To do this, we can use depth-first search (DFS) to try every path as well as return `None` if we come across a cycle. The [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) module in Python is quite handy in this case:

In [1]:
from collections import Counter

def max_path(s, lst):
    adj = [[] for v in s]
    
    # Build adjacency list
    for u, v in lst:
        adj[u].append(v)
    
    maximum_path = 0
    
    # Try every path from node `v`
    for v in range(len(s)):
        # Every item in this stack has the form: (path_string, visited, current_node)
        stack = [(s[v], set([v]), v)]
        
        while stack:
            path_string, visited, current_node = stack.pop()
            
            # Count value of current path and update maximum path if necessary
            cnt = Counter(path_string)
            _, path_val = cnt.most_common(1)[0]
            maximum_path = max(maximum_path, path_val)
            
            for neighbor in adj[current_node]:
                if neighbor in visited:
                    # There is a cycle
                    return None
                
                stack.append((path_string + s[neighbor], visited.union([neighbor]), neighbor))
    
    return maximum_path

This solution, however, would be terribly slow. DFS is `O(V+E)`, where `V` and `E` are the number of vertices and edges, respectively. Since we also evaluate the current path each time, our algorithm is `O(V*(V+E))`. Let's try to improve this runtime.

Notice that we're recomputing the whole path on each iteration. This is inefficient since, for example, only one character could change, so we should only need to increment that one character. This sounds like a good problem for dynamic programming.

Furthermore, notice that since we're using the alphabet of uppercase characters, we have a fixed number (26) of potential values that contribute to the longest chain.

Let's keep a matrix of size `N` by 26. `A[i][j]` will contain the maximum value of the path that can be made from the character `i` (where `i` will index into the alphabet, so `A=0`, `B=1`, etc. Then we'll use the following recurrence to keep track of the path with the largest value:
- When we get to a node `v`, we'll do DFS on all its neighbors
- Then, `A[v][j]` will be the maximum of all `A[neighbor][j]` for all its neighbors
- Then, we also need to count the current node too, so increment `A[v][current_char]` by one, where `current_char` is the current node's assigned letter

We will use DFS, like before, to actually search the graph as well as to determine if we have a cycle.

In [ ]:
VISITED = 0
UNVISITED = 1
VISITING = 2

def max_path(s, lst):
    adj = [[] for v in s]
    
    # Build adjacency list
    for u, v in lst:
        adj[u].append(v)
        
    # Create matrix cache
    dp = [[0 for _ in range(26)] for _ in range(len(s))]
    state = {v: UNVISITED for v in range(len(s))}
    
    # Define function for carrying out depth first search
    def dfs(v):
        state[v] = VISITING
        for neighbor in adj[v]:
            if state[neighbor] == VISITING:
                # We have a cycle
                return True
            
            dfs(neighbor)
            for i in range(26):
                dp[v][i] = dp[neighbor][i]
        
        current_char = ord(s[v]) - ord('A')
        dp[v][current_char] += 1
        state[v] = VISITED

    # Run DFS on graph
    for v in range(len(s)):
        if state[v] == UNVISITED:
            # The DFS function returns True if a cycle is found in the graph
            if dfs(v):
                return None
            
    return max(max(v for v in node) for node in dp)

Our revised solution will now run in just `O(V+E)` time, which is the same time complexity as standalone depth-first search.